# ACCIONES

In [ ]:
# ================================================
# Pair Trading en Colab (sin pip, 100% compatible)
# Fuente de datos: Stooq (CSV HTTP)
# ================================================

import io
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Intentar usar statsmodels (si no está, seguimos sin Engle–Granger)
try:
    import statsmodels.api as sm
    from statsmodels.tsa.stattools import coint
    HAS_SM = True
except Exception:
    HAS_SM = False
    sm = None

In [ ]:
plt.rcParams["figure.figsize"] = (10, 4.5)
plt.rcParams["axes.grid"] = True
pd.set_option("display.float_format", lambda x: f"{x:,.6f}")

In [ ]:
# ------------------------------
# 1) Loader robusto desde Stooq
# ------------------------------
def stooq_url(ticker: str, interval: str = "d") -> str:
    """
    Construye la URL CSV de Stooq para un 'ticker' (ej. 'aapl', 'msft').
    interval: 'd' diario, 'w' semanal, 'm' mensual.
    """
    # Stooq usa minúsculas; tickers USA típicos: AAPL -> aapl
    t = ticker.strip().lower()
    return f"https://stooq.com/q/d/l/?s={t}&i={interval}"

def fetch_stooq_series(ticker: str, start=None, end=None, tries: int = 3, pause: float = 1.5) -> pd.Series:
    """
    Descarga Close diario de un ticker desde Stooq, con reintentos.
    Devuelve una Serie indexada por fecha (datetime) con nombre = ticker en MAYÚSCULA.
    """
    last_err = None
    url = stooq_url(ticker, "d")
    for k in range(tries):
        try:
            s = pd.read_csv(url)
            # Estructura esperada: Date,Open,High,Low,Close,Volume
            if "Date" in s.columns and "Close" in s.columns:
                s["Date"] = pd.to_datetime(s["Date"], errors="coerce", utc=False)
                s = s.dropna(subset=["Date"]).sort_values("Date")
                s = s.set_index("Date")["Close"].astype(float)
                if start is not None:
                    s = s[s.index >= pd.to_datetime(start)]
                if end is not None:
                    s = s[s.index <= pd.to_datetime(end)]
                s.name = ticker.upper()
                if not s.empty:
                    return s
        except Exception as e:
            last_err = e
        time.sleep(pause * (k + 1))
    raise RuntimeError(f"Stooq fallo para {ticker}. Último error: {last_err}")

def load_prices_stooq(tickers, start=None, end=None) -> pd.DataFrame:
    """
    Descarga 'Close' para varios tickers desde Stooq y devuelve
    un DataFrame (columnas = tickers en MAYÚSCULA).
    """
    series = []
    for t in tickers:
        s = fetch_stooq_series(t, start=start, end=end)
        series.append(s)
    df = pd.concat(series, axis=1)
    # Stooq a veces tiene huecos no simultáneos: forward-fill y limpiar faltantes iniciales
    df = df.ffill().dropna(how="any")
    return df

### CORTADO CONTINUAMOS CLASE 5.